In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

In [3]:
%cd "Documents/GitHub/pmx_gasoline_theft_prediction/"

/Users/jorgeroa/Documents/GitHub/pmx_gasoline_theft_prediction


In [8]:
df_density_km = pd.read_excel("data-raw/BASE-INFRAESTRUCTURA.xlsx", 'DATOS')
df_density_km_w = df_density_km.drop(['CARR-MT', "RED-MT", "AREAMT ", "NOM_MUN"], axis=1)



df_density_km_w

,CVEMUN,CARR-KM,LINCARR,RED-KM,LINRED,AREAKM,DENSCARR,DENSRED
0,1001,438.190142,1053,765.173487,5500,1178.10,0.371946,0.649498
1,1002,173.662987,284,158.986649,615,549.01,0.316320,0.289588
2,1003,114.439835,207,193.630319,598,932.74,0.122692,0.207593
3,1004,69.959236,108,28.910152,255,129.74,0.539226,0.222831
4,1005,151.549502,438,177.436744,1292,504.99,0.300104,0.351367
...,...,...,...,...,...,...,...,...
2453,32054,78.565881,72,60.179296,249,375.36,0.209308,0.160324
2454,32055,249.744277,318,130.004540,627,2179.16,0.114606,0.059658
2455,32056,125.274509,299,132.960053,1698,441.50,0.283747,0.301155
2456,32057,79.643620,84,13.628317,195,220.81,0.360688,0.061720


In [10]:
dict_var_density = {"CVEMUN": "CVEGEO"}


df_density_km_w = df_density_km_w.rename(columns = dict_var_density)

df_density_km_w['CVEGEO'] = df_density_km_w['CVEGEO'].astype(str).str.zfill(5)
df_density_km_w

,CVEGEO,CARR-KM,LINCARR,RED-KM,LINRED,AREAKM,DENSCARR,DENSRED
0,01001,438.190142,1053,765.173487,5500,1178.10,0.371946,0.649498
1,01002,173.662987,284,158.986649,615,549.01,0.316320,0.289588
2,01003,114.439835,207,193.630319,598,932.74,0.122692,0.207593
3,01004,69.959236,108,28.910152,255,129.74,0.539226,0.222831
4,01005,151.549502,438,177.436744,1292,504.99,0.300104,0.351367
...,...,...,...,...,...,...,...,...
2453,32054,78.565881,72,60.179296,249,375.36,0.209308,0.160324
2454,32055,249.744277,318,130.004540,627,2179.16,0.114606,0.059658
2455,32056,125.274509,299,132.960053,1698,441.50,0.283747,0.301155
2456,32057,79.643620,84,13.628317,195,220.81,0.360688,0.061720


In [11]:
df_density_km_w.to_excel('data/df_density_km.xlsx', index = False)
df_density_km_w.to_parquet('data/df_density_km.parquet')

In [30]:
pipe_thefts = pd.read_excel("data/df_pipethefts_15_19.xlsx")

dict_var_density = {"Mun_code": "CVEGEO"}


pipe_thefts = pipe_thefts.rename(columns = dict_var_density)

pipe_thefts['CVEGEO'] = pipe_thefts['CVEGEO'].astype(str).str.zfill(5)


pipe_thefts


pipe_thefts_final = pd.merge(df_density_km_w,
                             pipe_thefts,
                             on='CVEGEO',
                             how='outer',
                             indicator=True)

pipe_thefts_final["Num_thefts"] = pipe_thefts_final["Num_thefts"].fillna(0)

pipe_thefts_final['CVEGEO'] = pipe_thefts_final['CVEGEO'].astype(str).str.zfill(5)


pipe_thefts_final.to_csv('data/df_pipe_CVEGEO.csv')

pipe_thefts_final

,CVEGEO,CARR-KM,LINCARR,RED-KM,LINRED,AREAKM,DENSCARR,DENSRED,Num_thefts,_merge
0,01001,438.190142,1053,765.173487,5500,1178.10,0.371946,0.649498,8.0,both
1,01002,173.662987,284,158.986649,615,549.01,0.316320,0.289588,0.0,left_only
2,01003,114.439835,207,193.630319,598,932.74,0.122692,0.207593,0.0,left_only
3,01004,69.959236,108,28.910152,255,129.74,0.539226,0.222831,0.0,left_only
4,01005,151.549502,438,177.436744,1292,504.99,0.300104,0.351367,0.0,left_only
...,...,...,...,...,...,...,...,...,...,...
2453,32054,78.565881,72,60.179296,249,375.36,0.209308,0.160324,0.0,left_only
2454,32055,249.744277,318,130.004540,627,2179.16,0.114606,0.059658,0.0,left_only
2455,32056,125.274509,299,132.960053,1698,441.50,0.283747,0.301155,0.0,left_only
2456,32057,79.643620,84,13.628317,195,220.81,0.360688,0.061720,0.0,left_only
